In [1]:
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu126
!pip install --upgrade transformers accelerate bitsandbytes
!pip install -q "langchain>=0.2.10,<1.0.0" "langchain-community>=0.2.10" \
                "langchain-text-splitters>=0.2.0" "chromadb>=0.5.5" \
                "sentence-transformers>=2.2.2" "pypdf>=4.2.0"

Looking in indexes: https://download.pytorch.org/whl/cu126


In [2]:
!pip install faiss-cpu pdfplumber

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import random
import numpy as np
import os
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
import faiss
import pdfplumber
import math
import re
import json

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
DATA_DIR = "/content/extracted"

In [7]:
import zipfile

zip_path = "/content/drive/MyDrive/data.zip"        # path to your zip file
extract_path = DATA_DIR  # where to extract

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [8]:
DATA_DIR = "/content/extracted/data/files/no_take_home"
QA_DIR = "/content/extracted/data/qa_pairs"

In [9]:
PERSIST_DIR_CARD = "./RAG_CARDIOLOGY"
COLLECTION_CARD = "hallucination_eval"
CHUNK_SIZE = 500
CHUNK_OVERLAP = 20

In [10]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document

def create_RAG(doc_dir, persist_dir, collection, chunk_size, chunk_overlap, emb):
  documents = []
  for d in os.listdir(doc_dir):
    path = os.path.join(doc_dir, d)
    loader = PyPDFLoader(path)
    pages = loader.load()

    # merge pages into one big Document
    full_text = "\n".join([p.page_content for p in pages])
    documents.append(Document(page_content=full_text, metadata={"source": path}))

  print(len(documents), "combined documents")
  splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    add_start_index=True,
  )
  chunks = splitter.split_documents(documents)
  print(f"Created {len(chunks)} chunks")

  vs = Chroma.from_documents(
      documents=chunks,
      embedding=emb,
      persist_directory=persist_dir,
      collection_name=collection,
      collection_metadata={"hnsw:space": "cosine"},
  )
  vs.persist()
  print("Vector count:", vs._collection.count())
  return vs

In [12]:
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipython-input-1019173367.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or dataset

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
vs_cardiology_fixed = create_RAG(DATA_DIR, PERSIST_DIR_CARD, COLLECTION_CARD, CHUNK_SIZE, CHUNK_OVERLAP, emb)
vs_cardiiology_student = create_RAG(DATA_DIR, PERSIST_DIR_CARD + "_student", COLLECTION_CARD + "_student", CHUNK_SIZE, CHUNK_OVERLAP, emb)

10 combined documents
Created 8380 chunks


/tmp/ipython-input-1866530331.py:31: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vs.persist()


Vector count: 8380
10 combined documents
Created 8380 chunks
Vector count: 8380


## Load the models

In [14]:
from google.colab import userdata
key = userdata.get('hf_key')
model_name = "meta-llama/Llama-2-13b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16"
)

# Explicitly tell the tokenizer to use the SentencePiece model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, use_auth_token=key)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    use_auth_token=key
)

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1041: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

# Setup contrastive decoding methods

In [15]:
import torch
import torch.nn.functional as F
from transformers import GenerationConfig

# --- Helper function to get logits ---
@torch.inference_mode()
def get_logits(model, input_ids):
    """
    Returns the logits for the last token of input_ids.
    """
    outputs = model(input_ids)
    return outputs.logits[:, -1, :]  # shape: [batch, vocab_size]

# --- Mask tokens below alpha fraction of max probability ---
def top_prob_mask_from_logits(logits, alpha=0.1):
    """
    Returns a boolean mask where tokens with probability >= alpha * max_prob are True.
    """
    probs = F.softmax(logits, dim=-1)
    max_p = probs.max(dim=-1, keepdim=True).values
    mask = probs >= (max_p * alpha)
    return mask

# --- Main contrastive decoding function ---
def contrastive_decoding(
    prompt_base, prompt_anti, tokenizer, base_model, anti_model,
    alpha=0.1, min_new_tokens=30, max_new_tokens=150
):
    device = base_model.device

    # Encode prompts
    inputs_base = tokenizer(prompt_base, return_tensors="pt").to(device)
    inputs_anti = tokenizer(prompt_anti, return_tensors="pt").to(device)

    # Initialize generation
    generated_base = inputs_base["input_ids"]
    generated_anti = inputs_anti["input_ids"]

    # Tracking outputs
    logprobs_list_expert = []
    ids_logprobs_expert = []
    contrastive_decoding_probs = []
    expert_top10_after_contrastive = []

    gen_config = GenerationConfig(
        min_new_tokens=min_new_tokens,
        max_new_tokens=max_new_tokens,
        do_sample=False
    )

    with torch.no_grad():
        for _ in range(gen_config.max_new_tokens):
            # Get logits for base and anti models
            expert_logits = get_logits(base_model, generated_base)
            anti_logits = get_logits(anti_model, generated_anti)

            # Expert probs and top-10 tokens
            expert_probs = F.softmax(expert_logits, dim=-1)
            topk_probs, topk_ids = torch.topk(expert_probs, k=10, dim=-1)

            logprobs_list_expert.append(topk_probs)
            ids_logprobs_expert.append(topk_ids)

            # Mask low-probability tokens
            mask = top_prob_mask_from_logits(expert_logits, alpha)

            # Contrastive scoring
            logp_expert = F.log_softmax(expert_logits, dim=-1)
            logp_anti = F.log_softmax(anti_logits, dim=-1)
            scores = logp_expert - logp_anti  # shape: [1, vocab_size]

            # Track scores of previous top-10 tokens
            expert_top10_after_contrastive.append(scores.gather(1, topk_ids))

            # Apply mask
            scores[~mask] = -1e9

            # Choose next token
            next_id = torch.argmax(scores, dim=-1).unsqueeze(0)  # shape: [1,1]

            # Append token to generated sequences
            generated_base = torch.cat([generated_base, next_id], dim=1)
            generated_anti = torch.cat([generated_anti, next_id], dim=1)

            # Track top-10 scores after contrastive step
            topk_probs_after, _ = torch.topk(scores, k=10, dim=-1)
            contrastive_decoding_probs.append(topk_probs_after)

            # Optional: stop if EOS token generated
            if next_id.item() == tokenizer.eos_token_id:
                break

    # Decode generated text (excluding prompt)
    generated_text = tokenizer.decode(
        generated_base[0, inputs_base["input_ids"].shape[1]:],
        skip_special_tokens=True
    )

    return generated_text, contrastive_decoding_probs, expert_top10_after_contrastive, logprobs_list_expert, ids_logprobs_expert


# Experiment 1: top 10 and top 10-20

In [23]:
def get_RAG(vector_db, query, k=14):
  rag_docs = vector_db.similarity_search(query, k=k)
  return rag_docs

def format_RAG_docs(docs):
  return "\n\n".join([
    f"Source: {doc.metadata['source']}\n{doc.page_content}"
    for doc in docs
  ])

def create_prompt(question, k, vector_db):
  rag_docs = get_RAG(vector_db, question, k=k)
  split_point = math.floor(len(rag_docs)/2)
  rag_docs_expert = rag_docs[:split_point]
  rag_docs_anti = rag_docs[split_point:]
  formatted_rag_docs_expert = format_RAG_docs(rag_docs_expert)
  formatted_rag_docs_anti = format_RAG_docs(rag_docs_anti)
  prompt_expert = f"""Using the following retrieved passages, answer the medical question concisely (1-2 sentences).
Format your answer exactly as:

Answer: <your concise answer>
Confidence: <X%>
Citation: <document/source>

Question: {question}

Passages:
{formatted_rag_docs_expert}

Answer:
  """
  prompt_anti = f"""Using the following retrieved passages, answer the medical question concisely (1-2 sentences).
Format your answer exactly as:

Answer: <your concise answer>
Confidence: <X%>
Citation: <document/source>

Question: {question}

Passages:
{formatted_rag_docs_anti}

Answer:
  """

  return prompt_expert, prompt_anti

In [17]:
generated_text, _, _, _, _ = contrastive_decoding(
    "We are never going home", "Until death do us part", tokenizer, model, model,
    alpha=0.1, min_new_tokens=30, max_new_tokens=150
)

In [20]:
import pandas as pd

In [21]:
def experiment_1(model, tokenizer, vector_db, doc_dir, k):
  results = []
  doc_names = os.listdir(doc_dir)
  doc_names = [doc.split(".")[0] for doc in doc_names]

  qa_objects = os.listdir(QA_DIR)
  for doc in doc_names:
    print(doc)
    qa_object = [qa for qa in qa_objects if doc + ".json" in qa][0]
    with open(os.path.join(QA_DIR, qa_object), "r") as f:
      data = json.load(f)

    questions = data["questions"]
    answers = data["answers"]

    for i, question in enumerate(questions):
      print(f"Question {i+1}/{len(questions)}")
      prompt_expert, prompt_base = create_prompt(question, k, vector_db)

      expert_text, contrastive_decoding_probs, expert_top10_after_contrastive,  logprobs_list_expert, ids_logprobs_expert = contrastive_decoding(
          prompt_expert, prompt_base, tokenizer, model, model,
          alpha=0.1, min_new_tokens=30, max_new_tokens=150
      )

      results.append({
          "id": f"{doc}_question_{i}",
          "response" : expert_text,
          "answer": answers[i],
          "contrastive_decoding_probs": contrastive_decoding_probs,
          "expert_top10_after_contrastive": expert_top10_after_contrastive,
          "logprobs_list_expert": logprobs_list_expert,
          "ids_logprobs_expert": ids_logprobs_expert
      })
  df = pd.DataFrame(results).set_index("id")
  return df

In [ ]:
df = experiment_1(model, tokenizer, vs_cardiology_fixed, DATA_DIR, 14)

doc7
Question 1/5
Question 2/5
Question 3/5
Question 4/5
Question 5/5
doc2
Question 1/5
Question 2/5
Question 3/5
Question 4/5
Question 5/5
doc1
Question 1/5
Question 2/5
Question 3/5
Question 4/5
Question 5/5
doc8
Question 1/5
Question 2/5
Question 3/5
Question 4/5
Question 5/5
doc5
Question 1/5
Question 2/5
Question 3/5
Question 4/5
Question 5/5
doc6
Question 1/5
Question 2/5
Question 3/5
Question 4/5
Question 5/5
doc9
Question 1/5
Question 2/5
Question 3/5


In [ ]:
df.to_csv("/content/drive/MyDrive/CD1_results_fixed.csv")

# Experiment 2

In [ ]:
def create_prompt_2(question, k, vector_db):
  rag_docs = get_RAG(vector_db, question, k=k)
  formatted_rag_docs = format_RAG_docs(rag_docs)
  prompt = f"""Using the following retrieved passages, answer the medical question concisely (1-2 sentences).
Format your answer exactly as:

Answer: <your concise answer>
Confidence: <X%>
Citation: <document/source>

Question: {question}

Passages:
{formatted_rag_docs}

Answer:
  """
  return prompt

In [ ]:
def delete_doc_from_vector_db(vector_db, doc):
  path = os.path.join(DATA_DIR, doc)
  chunks = vector_db.get(where={"source": path})
  vector_db.delete(where={"source": path})
  return chunks

def insert_chunks_into_vector_db(vector_db, chunks):
  docs_to_add = [
    Document(page_content=text, metadata=meta)
    for text, meta in zip(chunks['documents'], chunks['metadatas'])
  ]
  vector_db.add_documents(
      documents=docs_to_add,
      ids=chunks['ids']  # your existing IDs
  )

In [ ]:
from annotated_types import doc
def experiment_2(model, tokenizer, vector_db, mutable_db, doc_dir, k):
  results = []
  doc_names = os.listdir(doc_dir)
  doc_names = [doc.split(".")[0] for doc in doc_names]

  qa_objects = os.listdir(QA_DIR)
  for doc in doc_names:
    qa_object = [qa for qa in qa_objects if doc + ".json" in qa][0]
    with open(os.path.join(QA_DIR, qa_object), "r") as f:
      data = json.load(f)

    questions = data["questions"]
    answers = data["answers"]

    # delete the relevant doc out of the RAG
    chunks = delete_doc_from_vector_db(mutable_db, doc + ".pdf")

    for i, question in enumerate(questions):
      prompt_expert = create_prompt_2(question, k, vector_db)
      anti_prompt = create_prompt_2(question, k, mutable_db)
      expert_text, contrastive_decoding_probs, expert_top10_after_contrastive,  logprobs_list_expert, ids_logprobs_expert = contrastive_decoding(
          prompt_expert, anti_prompt, tokenizer, model
          alpha=0.1, min_new_tokens=30, max_new_tokens=150
      )

      results.append({
          "id": f"{doc}_question_{i}",
          "response" : expert_text,
          "answer": answers[i],
          "contrastive_decoding_probs": contrastive_decoding_probs,
          "expert_top10_after_contrastive": expert_top10_after_contrastive,
          "logprobs_list_expert": logprobs_list_expert,
          "ids_logprobs_expert": ids_logprobs_expert,
      })
    # reinsert the doc after
    insert_chunks_into_vector_db(mutable_db, chunks)

  df = pd.DataFrame(results).set_index("id")
  return df

In [ ]:
df = experiment_2(model, tokenizer, vs_cardiology_fixed, vs_cardiiology_student, DATA_DIR, 14)

In [ ]:
df.to_csv("/content/drive/MyDrive/CD2_results_held_out.csv")